In [1]:
from imports import genai, adk, BaseModel, Field

import os
print("📚 Imports loaded and Gemini auth is ready.")
print("CWD:", os.getcwd())

/Users/viveks/miniforge3/envs/adhd/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ imports.py: Authenticated with Google AI Studio and loaded ADK imports.
📚 Imports loaded and Gemini auth is ready.
CWD: /Users/viveks/Developer/adhd-assistant-capstone


In [2]:
# Quick sanity check: call Gemini once
model = genai.GenerativeModel("gemini-2.5-flash")

resp = model.generate_content("Say hi in one short sentence.")
print(resp.text)

Hi there!


In [3]:
# Cell 3: Import Agent Modules
# =================================================================
try:
    from agents import ConversationManagerAgent, TaskLogicAgent, ToolExecutionAgent
    print("✅ Successfully imported Agent classes.")
except ImportError as e:
    print(f"❌ Error importing agents: {e}")
    print("Make sure agents.py and tools.py are in the current folder.")

✅ Successfully imported Agent classes.


In [4]:
# Cell 4: Create Agent Instances
# =================================================================
# 1. Create the Specialist Agents
# Note: TaskLogicAgent now defaults to 'gemini-2.5-pro' per our refinement
task_agent = TaskLogicAgent()
tool_agent = ToolExecutionAgent()

# 2. Create the Supervisor (Conversation Manager)
# This injects the specialists into the coordinator
manager = ConversationManagerAgent(task_agent=task_agent, tool_agent=tool_agent)

print("✅ Agent System Online. Ready for user input.")

✅ Agent System Online. Ready for user input.


In [5]:
# Cell 5: Simulate User Interaction
# =================================================================

# 1. Define the User's "Brain Dump"
user_input = (
    "I have a big presentation on Friday morning at 10 AM. "
    "I need to call the doctor sometime next week to make an appointment, "
    "and I also have to remember to buy eggs and bread today."
)

print(f"👤 USER SAYS:\n{user_input}\n")
print("-" * 60)

# 2. Handle the message (with auto_confirm=False to test HITL)
# This mimics the "Think" step where the agent plans but waits for approval
agent_turn = manager.handle_user_message(
    user_text=user_input,
    user_id="notebook_test_user_01",
    auto_confirm=False
)

# 3. Display the Agent's proposed response
print(f"🤖 AGENT RESPONDS:\n{agent_turn.user_facing_message}")

👤 USER SAYS:
I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.

------------------------------------------------------------
🧠 [MEMORY] Reading profile for: notebook_test_user_01
🤖 AGENT RESPONDS:
Alex, you've done a fantastic job getting these thoughts out! It's completely normal for a 'big presentation' to feel like a lot, but by breaking it down into these smaller, actionable steps, you're already building a clear path forward. This approach directly supports your goal of improving executive function and will help reduce that anxiety around deadlines. Remember, each small task you complete is a win, bringing you closer to a successful Friday. You've got this, and I'm here to help you keep that momentum going!

Here's what I've broken down for you:
1. Outline and prepare the core content for the presentation. (Due: Thursday)
2. Create or finalize all prese

In [6]:
# Cell 6: User Confirmation & Tool Execution
# =================================================================

if agent_turn.requires_confirmation:
    print("✅ USER ACTION: Confirming plan...\n")

    # Execute the pending actions using the Tool Execution Agent
    # This mimics the "Act" step
    results = tool_agent.execute_actions(agent_turn.pending_actions)

    print("--- 🛠️ Tool Execution Results ---")
    for res in results:
        # Check for success/failure in the tool output
        status = res.get("status", "unknown")
        details = res.get("details", str(res))
        print(f"[{status.upper()}] {details}")

else:
    print("ℹ️ No actions required confirmation.")

✅ USER ACTION: Confirming plan...

📅 [CALENDAR] Scheduling 'Outline and prepare the core content for the presentation.'...
📅 [CALENDAR] Scheduling 'Create or finalize all presentation slides and visual aids.'...
📅 [CALENDAR] Scheduling 'Practice the presentation delivery at least once.'...
📅 [CALENDAR] Scheduling 'Confirm all technical requirements and logistics for the presentation (e.g., projector, microphone, room setup).'...
📅 [CALENDAR] Scheduling 'Call the doctor's office to schedule an appointment.'...
📅 [CALENDAR] Scheduling 'Buy eggs and bread.'...
--- 🛠️ Tool Execution Results ---
[SUCCESS] Scheduled 'Outline and prepare the core content for the presentation.' for Thursday. Total events: 14
[SUCCESS] Scheduled 'Create or finalize all presentation slides and visual aids.' for Thursday. Total events: 15
[SUCCESS] Scheduled 'Practice the presentation delivery at least once.' for Thursday. Total events: 16
[SUCCESS] Scheduled 'Confirm all technical requirements and logistics for 